# Assignment #4 - Reinforcement Learning

### <font color="red"> DUE: Nov 26 (Tuesday) 11:00 pm </font>  

<font color="blue"> Abhilash Mandlekar </font>

# I. Overview

Describe the objective of this assignment. You can briefly state how you accompilsh it.

# II. Problems 



## Rummy (Our Way)
![](https://upload.wikimedia.org/wikipedia/commons/9/9a/3_playing_cards.jpg)



### Description:
Rummy is mostly played by 4 players where each player get 10 cards and rest of placed as closed deck.  

But for this assignment to keep it simple, we have changed some rules, reduced the deck of cards and distribution of the cards among each player. Otherwise, the our Q table would be large to handle on our local machines.  

So in this game each player will get 3 cards randomly from 'A' to '5'. With each turn you have to pick a card either from the pile of closed or open cards depending on the cards you have. After that you have to select a card to drop to the pile from your deck. Your goal is to make all of your 3 cards of same number. As soon as one player reaches the goal state, it will meld the cards and game will stop and all other player will get score as the sum of their cards in hand. Player with minimum score will win the game.  

Each player will play for the 10 rounds and if nobody reach to the meld stage then each will player will get score equivalent to the sum of their cards in hand.

Player with the lowest score wins the game.



### <font color="red"> Game Introduction and Rules: </font>

You can only till max turns which is now 20. If you are able to meld your cards, you win otherwise you lose points as per your stash score.

Lets Simulate a Game Round :
The Goal of this game is to collect cards with the same rank.
Naming Conventions to follow H : Hearts, S : Spades, D : Diamonds, C : Clubs. Stash/Hand are the cards which you have. Stash/Hand Score: The sum of the rank of cards. 

- On the first turn you will be given 3 cards from the randomly shuffled deck. Eg: 3H, 2S, AD. Stash Score: 6
- In the next step you will have to either pick a card from pile or the deck. On the first turn, the top card of the deck is on the pile. The card on the pile is visible to you. So Card on the Pile : 2D . You have to decide whether you have to pick this card or you have to pick from pile. Since you alread
- Following the pickup step, you will have to drop one card from the stash/hand. Your Hand : 3H, 2S, AD, 2D
- You can drop any of these cards, but remember if you drop 2S or 2D you might not be able to make a series. So you have to decide whether to drop 3H or AD. Since there are maximum 20 turns, you have to keep your score low as well, Higher the score the higher points you lose. So you decide to drop 3H. Your hand/stash will be: 2S, 2D, AD
- Your opponent will play his hand, he/she shall pick up a card from a pile or deck and the drop a card to the pile.
- Now its your turn. The card on the Pile : 5H. Since picking up this card won't help you much. You decide to pick a card from the Deck. You get lucky, you get a 2C card. Your hand now is 2S, 2D, 2C, AD. Hurray! You can now meld these cards and drop the last card to make your score 0.
- You win this round. Your opponent Stash/Hand score : 15. So you are at 0 points and your opponent is at 15 points. You are ahead of him since you have to keep your score as low as possible.


## Rummy Environment

Importing Libraries

In [1]:
import random 
from functools import reduce
from collections import defaultdict
import numpy as np
from copy import copy
%matplotlib inline

Here, we define the SUITS, RANKS of the cards and their RANK_VALUE.

In [2]:
SUIT = ['H','S','D','C']
RANK = ['A', '2', '3', '4', '5','6','7']
RANK_VALUE = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'Q': 10, 'K': 10}

## Card Class Definition
__init__  : Defines the card details such as rank, suit and calculates the rank value


In [3]:
class Card :
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.rank_to_val = RANK_VALUE[self.rank]
    
    def __str__(self):
        return f'{self.rank}{self.suit}'

    def __repr__(self):
        return f'{self.rank}{self.suit}'

    
    def __eq__(self,other):
        return self.rank == other.rank and self.suit == other.suit
  
# Deck class contains some basic operations performed with the cards:
# 1. Shuffling the cards.
# 2. Drawing card from the deck.

## Deck Class Definition
__shuffle__ : Shuffles the deck in random order

__draw_card__ : Draws a card from the top of the deck

In [4]:
class Deck:
    def __init__(self,packs):
        self.packs = packs
        self.cards = []
        for pack in range(0,packs) :
            for suit in SUIT :
                for rank in RANK :
                    self.cards.append(Card(rank,suit))
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def draw_card(self):
        card = self.cards[0]
        self.cards.pop(0)
        return card

## Player Class:

### 1.__init__(self,name,stash=list(),isBot=False): 
Initializing stash, name, isBot/dealer points for each player.

### 2. deal_card(self,card):
This method appends the card in the stash and check the condition that length of stash should not be greater than nuber of cards length in game.

### 3. drop_card(self,card):
This method removes the card from stash and add that card into pile.

### 4. meld(self):
This method tries to find the cards with the same rank in the hand. If it finds then it will merge the cards in the hand to the melded cards array in the game. 

### 5. stash_score(self):
This method calculates sum of all the cards in stash according to the rank of each card.

### 6. get_info(self,debug):
This function fetch all the information of the player.

In [5]:
class Player:
  
    """  
        Player class to create a player object.
        eg: player = Player("player1", list(), isBot = False)
        Above declaration will be for your agent.
        All the player names should be unique or else you will get error.
        
    """

    def __init__(self,name,stash=list(),isBot=False,points=0,conn=None):
        self.stash = stash
        self.name = name
        self.game = None
        self.isBot = isBot
        self.points = points
        self.conn = conn
        
    def deal_card(self,card):
        try :
            self.stash.append(card)
            if len(stash) > self.game.cardsLength + 1 :
                raise ValueError('Cannot have cards greater than ')
        except ValueError as err:
            print(err.args)
            
    def drop_card(self,card):
        self.stash.remove(card)
        self.game.add_pile(card)
        return -1

    
    def meld(self):
        card_hash = defaultdict(list)
        for card in self.stash:
            card_hash[card.rank].append(card)
        melded_card_ranks = []
        for (card_rank,meld_cards) in card_hash.items():
            if len(meld_cards) >= 3 :
                self.game.meld.append(meld_cards)
                melded_card_ranks.append(card_rank)
                for card in meld_cards:
                    self.stash.remove(card)
        
        for card_rank in melded_card_ranks :
            card_hash.pop(card_rank)
        return len(melded_card_ranks) > 0

    def stash_score(self) :
        score = 0
        for card in self.stash :
            score += RANK_VALUE[card.rank]
        return score
    
    def get_info(self,debug):
        if debug :
            print(f'Player Name : {self.name} \n Stash Score: {self.stash_score()} \n Stash : {", ".join(str(x) for x in self.stash)}')
        card_ranks = []
        card_suits = []
        pileset = None
        pile = None
        for card in self.stash :
            card_suits.append(RANK_VALUE[card.rank])
            card_ranks.append(card.suit)
        if len(self.game.pile) > 0 : 
            return {"Stash Score" : self.stash_score(), "CardSuit":  card_suits, "CardRanks": card_ranks, "PileRank": self.game.pile[-1].rank, "PileSuit":self.game.pile[-1].suit}
        return {"Stash Score" : self.stash_score(), "CardSuit":  card_suits, "CardRanks": card_ranks}


## Game Enviroment:

### 1. __init():  

### 2. add_pile(self, card):  This method takes a card as argument and first checks number of cards in the deck. If its is ‘0’ then add the cards from file to deck and append the passed card to the pile.

### 3. pick_card(self, player, action):  This methods helps player picking up the card from either Pile or Deck based on action.
			We have defined, If action = 0 then, player will pick a card from Pile
			And if action. = 1 then player will pick card from Deck. 
			Meld condition will be check after player picks the card, if the meld condition satisfied, player won. 
            You can modify the rewards in return, only the values

### 4. pick_from_pile(self, player): This method helps player picking card from the pile and simultaneously a card from pile gets reduced.

### 5. pick_from_deck(self, player): This method is similar to above method but it helps picking up card from the Deck. 

### 6. get_player(self, player_name): This function fetch the details of the player given player_name.

### 7. computer_play(self, player): This method defines the play of the computer/Dealer in following sequence:
			--> Randomly taking actions from picking up card from deck/pile.
			--> Checking the meld condition afterwards.
			--> If the meld condition does not satisfied, remove the card from his stash.

### 8. play(self): This method defines all the function city of play for the player:
		  --> Decrementing the maximum number of turns defined per game.
		  --> For each player, it will check the 'stash', if the 'Stash' for any player = 0 (That player won), it will add the value of each card in stash for all other players.
		  --> Or If maximum number of turns in each round becomes 0, it will add the card values in stash for all the players and return.
      
### 9. drop_card(self,player,card) This method drops a card from the player given player object and card object and returns the reward associated with it.

      --> You can modify the rewards which gets returned in the value.


### 9. reset(self, players): This method will reinitialize all the deck, pile and players.

### 10. _update_turn(self): This method is used to count the number of turns in the game.

In [6]:
class RummyAgent() :
    """
    Simple Rummy Environment
    
    Simple Rummy is a game where you need to make all the cards in your hand same before your opponent does.
    Here you are given 3 cards in your hand/stash to play.
    For the first move you have to pick a card from the deck or from the pile. 
    The card in deck would be random but you can see the card from the pile.
    In the next move you will have to drop a card from your hand.
    Your goal is to collect all the cards of the same rank. 
    Higher the rank of the card, the higher points you lose in the game. 
    You need to keep the stash score low. Eg, if you can AH,7S,5D your strategy would be to either find the first pair of the card or by removing the highest card in the deck.
    You only have 20 turns to either win the same or collect low scoring card.
    You can't see other players cards or their stash scores.
    
    Parameters
    ====
    players: Player objects which will play the game.
    max_card_length : Number of cards each player can have
    max_turns: Number of turns in a rummy game
    """

    def __init__(self,players,max_card_length=5,max_turns=20) :
        self.max_card_length = max_card_length
        self.max_turns = max_turns
        self.reset(players)
        
    def update_player_cards(self,players):
        for player in players :
            player = Player(player.name,list(),isBot=player.isBot,points=player.points,conn=player.conn)
            stash = []
            for i in range(self.max_card_length):
                player.stash.append(self.deck.draw_card())
            player.game = self
            self.players.append(player)
        self.pile = [self.deck.draw_card()]

    def add_pile(self,card):
        if len(self.deck.cards) == 0 :
            self.deck.cards.extend(self.pile)
            self.deck.shuffle()
            self.pile = []
        self.pile.append(card)
        
        
    def pick_card(self,player,action):
        if action == 0:
            self.pick_from_pile(player)
        else :
            self.pick_from_deck(player)
        if player.meld() :
            return {"reward" : 10}
        else :
            return {"reward" : -1}
#             return -player.stash_score()
       
    def pick_from_pile(self,player):
        card = self.pile[-1]
        self.pile.pop()
        return player.stash.append(card)
     
    def pick_from_deck(self,player):
        return player.stash.append(self.deck.draw_card())
    
    def get_player(self,player_name):
        return_player = [player for player in self.players if player.name == player_name]
        if len(return_player) != 1:
            print("Invalid Player")
            return None
        else:
            return return_player[0]
    
    def drop_card(self,player,card):
        player.drop_card(card)
        return {"reward" : -1}
    
  
    def computer_play(self,player):
        #Gets a card from deck or pile
        if random.randint(0,1) == 1 :
            self.pick_from_pile(player)
        else :
            self.pick_from_deck(player)
            
        #tries to meld if it can
#         if random.randint(0,10) > 5 :
        player.meld()
        
        #removes a card from the stash
        if len(player.stash) != 0:
            card = player.stash[(random.randint(0,len(player.stash) - 1))]
            player.drop_card(card)
        
    def play(self):
        for player in self.players :
            if len(player.stash) == 0 :
                return True
        if self.max_turns <= 0 :
            return True
        return False

    def _update_turn(self):
        self.max_turns -= 1  
    
    def reset(self,players,max_turns=20):
        self.players = []
        self.deck = Deck(1)
        self.deck.shuffle()
        self.meld = []
        self.pile = []
        self.max_turns = max_turns
        self.update_player_cards(players)




### Example Run with Random Players

This following code shows how to create an environment and players to play 10 games with 20 maximum rounds. This only includes random picking up of card and dropping.  

In [7]:
p1 = Player('tabish',list())
p2 = Player('comp1',list(),isBot=True)
rummy = RummyAgent([p1,p2],max_card_length=3,max_turns=20)

In [8]:
maxiter = 3
debug = True
for j in range(maxiter):
        for player in rummy.players :
            player.points = player.stash_score()
        
        rummy.reset(rummy.players)
        random.shuffle(rummy.players)
        # int i = 0 
        if debug :
            print(f'**********************************\n\t\tGame Starts : {j}\n***********************************')
        while not rummy.play() :    
            rummy._update_turn()
            print(rummy.max_turns)       
            for player in rummy.players:
                if player.isBot :
                    if rummy.play():
                        continue
                    if debug :
                        print(f'{player.name} Plays')
                    rummy.computer_play(player)
                    if debug :
                        player.get_info(debug)
                        if player.stash == 0 :
                            print(f'{player.name} wins the round')

                else :
                    if rummy.play() :
                        continue
                    if debug :
                        print(f'{player.name} Plays')
                    player_info = player.get_info(debug)
                    action_taken = np.random.choice(1)
                    if debug :
                        print(f'Card in pile {player_info["PileSuit"]}{player_info["PileRank"]}')
                    result_1 = rummy.pick_card(player,action_taken)
                    result_1 = result_1["reward"]

                    if debug : 
                        print(f'{player.name} takes action {action_taken}')
                    #player stash will have no cards if the player has melded them 
                    #When you have picked up a card and you have drop it since the remaining cards have been melded. 
                    if len(player.stash) == 1:
                        rummy.drop_card(player,player.stash[0])
                        if debug :
                            print(f'{player.name} Wins the round')

                    elif len(player.stash) != 0 :

                        player_info = player.get_info(debug)
                        s = player_info['CardRanks']
                        action_taken = np.random.choice(4)
                        card = player.stash[action_taken]
                        if debug :
                            print(f'{player.name} drops card {card}')

                        result_1 = rummy.drop_card(player,card)
                        result_1 = result_1["reward"]
#                             pdb.set_trace()
                    else :
                        if debug :
                            print(f'{player.name} Wins the round')
                    if debug :
                        player.get_info(debug)
        


**********************************
		Game Starts : 0
***********************************
19
tabish Plays
Player Name : tabish 
 Stash Score: 9 
 Stash : AD, 5D, 3S
Card in pile H7
tabish takes action 0
Player Name : tabish 
 Stash Score: 16 
 Stash : AD, 5D, 3S, 7H
tabish drops card 7H
Player Name : tabish 
 Stash Score: 9 
 Stash : AD, 5D, 3S
comp1 Plays
Player Name : comp1 
 Stash Score: 17 
 Stash : 6S, 4D, 7H
18
tabish Plays
Player Name : tabish 
 Stash Score: 9 
 Stash : AD, 5D, 3S
Card in pile D3
tabish takes action 0
Player Name : tabish 
 Stash Score: 12 
 Stash : AD, 5D, 3S, 3D
tabish drops card 5D
Player Name : tabish 
 Stash Score: 7 
 Stash : AD, 3S, 3D
comp1 Plays
Player Name : comp1 
 Stash Score: 17 
 Stash : 6S, 4D, 7H
17
tabish Plays
Player Name : tabish 
 Stash Score: 7 
 Stash : AD, 3S, 3D
Card in pile D5
tabish takes action 0
Player Name : tabish 
 Stash Score: 12 
 Stash : AD, 3S, 3D, 5D
tabish drops card 3D
Player Name : tabish 
 Stash Score: 9 
 Stash : AD, 3S, 5

# III. Methods

- Decide your TD learning approach: SARSA or Q-learning? 
- Decide your function approximator.
- Describe your approach and the reason why you select it.
- Finish epsilon_greedy function and other TODOs. Explain it.
- Explain your codes.

## Review of SARSA and Q-learning

### SARSA Learning
SARSA is an on-policy algorithm where, in the current state, S an action, A is taken and the agent gets a reward, R and ends up in next state, S1 and takes action, A1 in S1.


#### Updation of Q-table
$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha ( R_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t))$$
#### Algorithm
    Initialize Q(s, a) arbitrarily
    REPEAT (for each episode):
        Initialize s
        Choose a from s using policy derived from Q
        Repeat (for each step of episode):
            Take action a, observe r, s'
            Choose a' from s' using policy derived from Q
            Update the Q-table as per the formula given above for SARSA learning
            s <-- s'
            a <-- a'
        until s is terminal


### Q Learning

Q-learning attempts to learn the value of being in a given state, and taking the specific action.<br>

Q-Learning is a table of values for every state (row) and action (column) possible in the environment. Within each cell of the table, we learn a value for how good it is to take a given action within a given state. <br>

We start by initializing the table to be uniform (all zeros), and then as we observe the rewards we obtain for various actions, we update the table accordingly.

We make updates to our Q-table using the Bellman equation, which states that the expected long-term reward for a given action is equal to the immediate reward from the current action combined with the expected reward from the best future action taken at the following state. In this way, we reuse our own Q-table when estimating how to update our table for future actions.<br>

#### Updation of Q-table in Q-learning
$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha ( R_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t))$$

#### Algorithm

    Initialize Q(s, a) arbitrarily
    REPEAT (for each episode):
        Initialize s
        Repeat (for each step of episode):
            Choose a from s using policy derived from Q
            Take action a, observe r, s'
            Update the Q-table as per the formula given above for Q-learning
            s <-- s'
        until s is terminal

## Choice of TD learning and reason

SARSA chooses an action following the same current policy and updates its Q-values whereas Q-learning chooses the greedy action, that is, the action that gives the maximum Q-value for the state.<br>

Since the agent should learn the optimal policies as well as explore all the possibilities, it is always a good idea to use Q-learning over SARSA learning.<br>

SARSA only looks up the next policy value while Q-learning looks up the next maximum policy value. Q-learning has the capabilitiy to continue learning while changing policies.

## Choice of Function Approximation and Reason

Picking an action can be simple by selecting one with maximum Q value. 

$$
a^* = \arg \max_a Q(S_t, a)
$$

However, this can cause limited experience to develop good Q estimation, and eventually a good policy. 
Without new data, greedy action selection will repeat the same actions, or repeatedly *exploit* your current knowledge. Thus, you need to *explore* other non-greedy actions to increase the experience to improve the Q estimation. 

This is called "exploration-exploitation dilemma." 


One of the way for this dilemma is $\epsilon$-greedy action selection. With a parameter $\epsilon \in [0, 1]$, we can control the exploration and exploitation level. When $\epsilon = 0$, the actions are selected in greedy manner, but when $\epsilon = 1$, the actions are selected randomly. 

## RLAgent for Rummy

Referencing the RLAgent in the lecture note, make your own agent class for the game. 

In [9]:
player.stash

[]

In [116]:
class Rummy():
    
    def __init__(self, env):
        print(env.players[0].name)
        env.reset(env.players)
        random.shuffle(env.players)
        env._update_turn()
        #self.size = env.get_size()
        #self.n_a = len(env.get_actions())
        #self.update_cards = env.update_player_cards()
        self.p1 = env.get_player(env.players[0].name)
        
        #self.pck_card = env.pick_card(self.p1, 0)
        #self.drp_card = env.drop_card(self.p1, card)
        self.action_taken = 0
        self.action_taken1 = 0
        # self.Q table including the surrounding border
        #self.Q = np.zeros((self.size[0], self.size[1], self.n_a))
        self.Q = np.zeros((8, 8, 3))
        
    def greedy(self,Q, s):
        return np.argmax(Q[self.action_taken])  # greedy action selection

    def epsilon_greed(self, epsilon, s):
        # TODO: Finish this function
        #if np.random.rand() < epsilon:
        #    return np.random.randint(self.action_taken)
        #else:
        return self.greedy(self.Q, s)
        
    def train(self, env, **params):
        gamma = params.pop('gamma', 0.99)
        alpha = params.pop('alpha', 0.1)
        epsilon= params.pop('epsilon', 0.1)
        maxiter= params.pop('maxiter', 1000) 
        maxstep= params.pop('maxstep', 1000)
        
        for player in env.players :
            player.points = player.stash_score()
        
        for j in range(maxiter):
            env.reset(env.players)
            random.shuffle(env.players)
            # int i = 0 
            if debug :
                print(f'**********************************\n\t\tGame Starts : {j}\n***********************************')
            while not env.play() :    
                env._update_turn()
                print(env.max_turns)       
                for player in env.players:
                    if player.isBot :
                        if env.play():
                            continue
                        if debug :
                            print(f'{player.name} Plays')
                        env.computer_play(player)
                        if debug :
                            player.get_info(debug)
                            if player.stash == 0 :
                                print(f'{player.name} wins the round')

                    else :
                        if env.play() :
                            continue
                        if debug :
                            print(f'{player.name} Plays')
                        player_info = player.get_info(debug)                      
                        s = player_info['CardSuit'] #get current cards
                        action_taken = self.epsilon_greed(epsilon, s)
                        if debug :
                            print(f'Card in pile {player_info["PileSuit"]}{player_info["PileRank"]}')
                        result_1 = env.pick_card(player,action_taken)
                        result_1 = result_1["reward"]

                        if debug : 
                            print(f'{player.name} takes action {action_taken}')
                        #player stash will have no cards if the player has melded them 
                        #When you have picked up a card and you have drop it since the remaining cards have been melded. 
                        if len(player.stash) == 1:
                            env.drop_card(player,player.stash[0])
                            if debug :
                                print(f'{player.name} Wins the round')
                                return result_1

                        elif len(player.stash) != 0 :

                            player_info = player.get_info(debug)
                            s1 = player_info['CardSuit']
                            
                            action_taken1 = self.epsilon_greed(epsilon, s1) ## to do
                            print("State " , s1)
                            print("State Q:", self.Q)
                            self.Q[s[0], s[1], action_taken] += alpha * (result_1 + gamma * self.Q[s1[0], s1[1], action_taken1] -\
                                             self.Q[s[0], s[1], action_taken])
                            card = player.stash[action_taken]
                            if debug :
                                print(f'{player.name} drops card {card}')

                            result_1 = env.drop_card(player,card)
                            result_1 = result_1["reward"]
    #                             pdb.set_trace()
                        else :
                            if debug :
                                print(f'{player.name} Wins the round')
                        if debug :
                            player.get_info(debug)



In [126]:
p1 = Player('p3',list())
p2 = Player('comp3',list(),isBot=True)
rummy_env = RummyAgent([p1],max_card_length=3,max_turns=20)
rummy = Rummy(rummy_env)

p3


In [127]:
p1.stash

[]

In [128]:
rtrace = rummy.train(rummy_env,
                     gamma=0.99, 
        alpha=0.1,
        epsilon=0.1,
        maxiter=100, 
        maxstep=1000)

**********************************
		Game Starts : 0
***********************************
19
p3 Plays
Player Name : p3 
 Stash Score: 12 
 Stash : AH, 6H, 5C
Card in pile H7
p3 takes action 0
Player Name : p3 
 Stash Score: 19 
 Stash : AH, 6H, 5C, 7H
State  [1, 6, 5, 7]
State Q: [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 

State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.89640839  0.          0.        ]
  [-0.3994004   0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.499001    0.          0.        ]]

 [[ 0.          0.       

  [ 0.          0.          0.        ]]]
p3 drops card AS
Player Name : p3 
 Stash Score: 14 
 Stash : 6D, 5C, 3H
13
p3 Plays
Player Name : p3 
 Stash Score: 14 
 Stash : 6D, 5C, 3H
Card in pile SA
p3 takes action 0
Player Name : p3 
 Stash Score: 15 
 Stash : 6D, 5C, 3H, AS
State  [6, 5, 3, 1]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-1.19342195  0.          0.        ]
  [-0.3994004   0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994

  [ 0.          0.          0.        ]]]
p3 drops card 6C
Player Name : p3 
 Stash Score: 7 
 Stash : 2D, 3H, 2H
16
p3 Plays
Player Name : p3 
 Stash Score: 7 
 Stash : 2D, 3H, 2H
Card in pile C6
p3 takes action 0
Player Name : p3 
 Stash Score: 13 
 Stash : 2D, 3H, 2H, 6C
State  [2, 3, 2, 6]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-1.48954536  0.          0.        ]
  [-0.3994004   0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.399400

  [ 0.          0.          0.        ]]]
p3 drops card AH
Player Name : p3 
 Stash Score: 8 
 Stash : 5H, AC, 2H
17
p3 Plays
Player Name : p3 
 Stash Score: 8 
 Stash : 5H, AC, 2H
Card in pile HA
p3 takes action 0
Player Name : p3 
 Stash Score: 9 
 Stash : 5H, AC, 2H, AH
State  [5, 1, 2, 1]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.1         0.          0.        ]
  [-1.48954536  0.          0.        ]
  [-0.3994004   0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.499001 

...

  [-0.499001    0.          0.        ]]]
p3 drops card 3C
Player Name : p3 
 Stash Score: 11 
 Stash : 3D, 2D, 6C
11
p3 Plays
Player Name : p3 
 Stash Score: 11 
 Stash : 3D, 2D, 6C
Card in pile C3
p3 takes action 0
Player Name : p3 
 Stash Score: 14 
 Stash : 3D, 2D, 6C, 3C
State  [3, 2, 6, 3]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-1.3909363   0.          0.        ]
  [-0.89640839  0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.499001    0.          0.        ]
  [-3.4411493   0.          0.        ]
  [-0.3994004   0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.8964

  [-0.499001    0.          0.        ]]]
p3 drops card 3D
Player Name : p3 
 Stash Score: 14 
 Stash : 3H, 7H, 4D
11
p3 Plays
Player Name : p3 
 Stash Score: 14 
 Stash : 3H, 7H, 4D
Card in pile D3
p3 takes action 0
Player Name : p3 
 Stash Score: 17 
 Stash : 3H, 7H, 4D, 3D
State  [3, 7, 4, 3]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-1.3909363   0.          0.        ]
  [-1.3909363   0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.499001    0.          0.        ]
  [-3.4411493   0.          0.        ]
  [-0.3994004   0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.8964

  [-0.499001    0.          0.        ]]]
p3 drops card AC
Player Name : p3 
 Stash Score: 13 
 Stash : 7H, AH, 5D
14
p3 Plays
Player Name : p3 
 Stash Score: 13 
 Stash : 7H, AH, 5D
Card in pile CA
p3 takes action 0
Player Name : p3 
 Stash Score: 14 
 Stash : 7H, AH, 5D, AC
State  [7, 1, 5, 1]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-1.3909363   0.          0.        ]
  [-1.88299651  0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.598502    0.          0.        ]
  [-3.4411493   0.          0.        ]
  [-0.598502    0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-1.3909

  [-0.598502    0.          0.        ]]]
p3 drops card 6S
Player Name : p3 
 Stash Score: 15 
 Stash : AH, 7S, 7C
14
p3 Plays
Player Name : p3 
 Stash Score: 15 
 Stash : AH, 7S, 7C
Card in pile S6
p3 takes action 0
Player Name : p3 
 Stash Score: 21 
 Stash : AH, 7S, 7C, 6S
State  [1, 7, 7, 6]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-1.3909363   0.          0.        ]
  [-1.88299651  0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.99551198  0.          0.        ]
  [-3.4411493   0.          0.        ]
  [-0.99551198  0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-1.3909

  [-0.99551198  0.          0.        ]]]
p3 drops card 7C
Player Name : p3 
 Stash Score: 13 
 Stash : 6C, 2C, 5C
0
**********************************
		Game Starts : 29
***********************************
19
p3 Plays
Player Name : p3 
 Stash Score: 17 
 Stash : 7D, 3H, 7C
Card in pile S3
p3 takes action 0
Player Name : p3 
 Stash Score: 20 
 Stash : 7D, 3H, 7C, 3S
State  [7, 3, 7, 3]
State Q: [[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-1.3909363   0.          0.        ]
  [-1.88299651  0.          0.        ]
  [-0.3994004   0.          0.        ]
  [-0.99551198  0.          0.        ]
  [-3.4411493   0.          0.        ]


In [121]:
result_1

10

### Reference

1) Arthur Juliani - https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0

2) Lecture Notes: http://localhost:8888/notebooks/Notebook/Note-ReinforcementLearning.Sol.ipynb


# IV - Results

- Describe the choice of your parameters for $\gamma$, $\alpha$, and $\epsilon$. 
  - Run experiments to find good parameters
  - Show the experimental outputs to show the process of your selection
- Visualize the results and explain outputs (even from mine.)
  - Run the codes and tell me what you observe
  - Add more visualizations to enrich your explanation.

# V. Conclusions

Discuss the challenges or somethat that you learned. 
If you have any suggestion about the assignment, you can write about it. 

# Extra Credit for Tournament Top16!

- In two classes, we will have qualification and tournament.
- To participate the tournament, you need to use this [client](http://nbviewer.jupyter.org/url/webpages.uncc.edu/mlee173/teach/itcs6156/notebooks/assign/StudentClient.ipynb) codes.
- Locally you can run the [server](http://nbviewer.jupyter.org/url/webpages.uncc.edu/mlee173/teach/itcs6156/notebooks/assign/SERVER.ipynb) to verify your run. 

### Qualification Round 

- In qualification game, individual will play against a fixed policy agent on the server
  - You can access the server: TBA
  
  - Highly ranked (by the profit) will proceed to the tournament.
  - max. 32 players will be cut for main tournament (by 11 pm on Dec 1st).
  
### Tournament

- In tournament, 4 players will be play in each round.
  - In a table game, two best scoring players will proceed to next round
  - In final, the game will ends in 50 rounds and winner will be the one with lowest balance. 

## Grading

We will test your notebook with an additional grid.txt file. Please make sure to test in different maze input files. 

points | | description
--|--|:--
5 | Overview| states the objective and the appraoch 
35 | Methods | 
 |10| Review of the SARSA and Q-Learning
 | 5| Choice of TD learning and Reason
 | 5| Choice of Function Approximation and Reason
 |10| Implementation of the selected approach (RLAgent)
 | 5| explanation of the codes
45 | Results 
 |10| Reports the selected parameters 
 |15| Experimental outputs that show the choice of parameters. How do you choose them?
 |10| Output plots (5 for each)
 |10| reading and analysis of learning results and plots
10 | Participation to the tournament.
5 | Conclusions 
